In [1]:
#! git commit -m "added docu"

On branch modis_scripts
Your branch is up to date with 'origin/modis_scripts'.

Changes not staged for commit:
	modified:   get_modis_gdal_post06.sh

Untracked files:
	Untitled.ipynb
	../notebooks/untitled.txt

no changes added to commit


In [2]:
#!git stash

Saved working directory and index state WIP on modis_scripts: fbd3302 added bash scripts to download and remap MODIS data from tile server


In [ ]:
#git branch delete modis_scripts
#git push origin --delete modis_scripts
#git config --global user.email schweig@uw.edu 
#git config --global user.name axelschweiger
#git push origin --delete modis_scripts

In [3]:
# Clone the repository with a glib layer definition
#!git clone https://github.com/nasa-gibs/gibs-ml.git

Cloning into 'gibs-ml'...
remote: Enumerating objects: 326, done.
remote: Counting objects: 100% (326/326), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 326 (delta 146), reused 326 (delta 146), pack-reused 0
Receiving objects: 100% (326/326), 17.29 MiB | 31.73 MiB/s, done.
Resolving deltas: 100% (146/146), done.
Checking out files: 100% (55/55), done.


In [56]:
import sys
from datetime import datetime, time, date
import os
import subprocess
sys.path.append('/home/jovyan/overcast/gibs-ml')

In [13]:
sys.path

['/home/jovyan/overcast/modis_scripts',
 '/srv/conda/envs/notebook/lib/python37.zip',
 '/srv/conda/envs/notebook/lib/python3.7',
 '/srv/conda/envs/notebook/lib/python3.7/lib-dynload',
 '',
 '/srv/conda/envs/notebook/lib/python3.7/site-packages',
 '/srv/conda/envs/notebook/lib/python3.7/site-packages/IPython/extensions',
 '/home/jovyan/.ipython',
 'gib-ml',
 '../gib-ml',
 '../gib-ml',
 '/home/jovyan/overcast/gib-ml']

In [31]:
from gibs_layer import GIBSLayer

In [115]:
# this is from an example in the gibs-ml
layer_name='VIIRS_SNPP_CorrectedReflectance_BandsM11_I2_I1'
layer_name='MODIS_Terra_CorrectedReflectance_TrueColor'
layer = GIBSLayer.get_gibs_layer(layer_name)
epsg='3413'
start_date = datetime.strptime("2019-06-10", "%Y-%m-%d")
tile_resolution='250m'
datestring = start_date.strftime("%Y-%m-%d")
datestring
output_dir='/home/jovyan/shared/data-overcast/modis/'
output_dir='./'

In [116]:
layer.generate_xml(protocol="tms", epsg=epsg, tile_resolution=tile_resolution, datestring=datestring)
xml=layer.gibs_xml
# this seems to generate something that looks like and XML file. It seems to be a string
xml

'<GDAL_WMS><Service name="TMS"><ServerUrl>https://gibs.earthdata.nasa.gov/wmts/epsg3413/best/MODIS_Terra_CorrectedReflectance_TrueColor/default/{Time}/250m/${z}/${y}/${x}.jpg</ServerUrl></Service><DataWindow><UpperLeftX>-4194304</UpperLeftX><UpperLeftY>4194304</UpperLeftY><LowerRightX>4194304</LowerRightX><LowerRightY>-4194304</LowerRightY><TileLevel>5</TileLevel><TileCountX>2</TileCountX><TileCountY>2</TileCountY><YOrigin>top</YOrigin></DataWindow><Projection>EPSG:3413</Projection><BlockSizeX>512</BlockSizeX><BlockSizeY>512</BlockSizeY><BandsCount>3</BandsCount><Timeout>3000</Timeout></GDAL_WMS>'

In [118]:
xml = xml.replace("{Time}", datestring)
# IMPORTANT ON WINDOWS: REPLACE " with '
xml = xml.rstrip()  # remove all trailing whitespace
# infile = infile.replace("\"", "'")

# Build name of output file
#xml = "'" + xml + "'"  # add extra quotes around XML
with open("temp.xml", "w") as f:
    f.write(xml)
outfile = os.path.join(output_dir, 'modis_output_test.tif')

In [76]:
proj_str ="'+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-145 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'" 
te_str = '-800000 -2500000 1200000 -400000'
tr_str = '250 250'

In [52]:
# gdalwarp -overwrite -t_srs '+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-145 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs' -te -800000 -2500000 1200000 -400000 -tr 250 250 $xmlfile $outfile

In [119]:
cmd_list = ["gdalwarp","-overwrite","-t_srs",proj_str,"-te",te_str,"-tr",tr_str,"temp.xml", outfile]
cmd= ' '.join(cmd_list)
#cmd="'"+cmd+"'"
print(cmd)

gdalwarp -overwrite -t_srs '+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-145 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs' -te -800000 -2500000 1200000 -400000 -tr 250 250 temp.xml ./modis_output_test.tif


'ls -l'


In [139]:
#process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE,shell='True')
print(cmd)
process = subprocess.Popen(cmd,shell='True',stdout=subprocess.PIPE,stderr=subprocess.PIPE)
out,err = process.communicate()
print(process.returncode, out,err, )
print("Done Writing File :" outfile)

gdalwarp -overwrite -t_srs '+proj=stere +lat_0=90 +lat_ts=70 +lon_0=-145 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs' -te -800000 -2500000 1200000 -400000 -tr 250 250 temp.xml ./modis_output_test.tif
b'Creating output file that is 8000P x 8400L.\nProcessing temp.xml [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.\n'
here


In [141]:
out, err

(b'Creating output file that is 8000P x 8400L.\nProcessing temp.xml [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.\n',
 b'')

In [94]:
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)


FileNotFoundError: [Errno 2] No such file or directory: '/srv/conda/envs/notebook/bin/gdalwarp -h': '/srv/conda/envs/notebook/bin/gdalwarp -h'

In [ ]:
# Sample XML file used by my bash script.
# <GDAL_WMS>
#     <Service name="TMS">
#         <ServerUrl>http://map1.vis.earthdata.nasa.gov/wmts-arctic/MODIS_Terra_CorrectedReflectance_REPLACE_THIS_STRING_WITH_TYPE/default/REPLACE_THIS_STRING_WITH_DATE/EPSG3413_250m/${z}/${y}/${x}.jpg</ServerUrl>
#     </Service>
#     <DataWindow>
#         <UpperLeftX>-4194304</UpperLeftX>
#         <UpperLeftY>4194304</UpperLeftY>
#         <LowerRightX>4194304</LowerRightX>
#         <LowerRightY>-4194304</LowerRightY>
#         <TileLevel>5</TileLevel>
#         <TileCountX>2</TileCountX>
#         <TileCountY>2</TileCountY>
#         <YOrigin>top</YOrigin>
#     </DataWindow>
#     <Projection>EPSG:3413</Projection>
#     <BlockSizeX>512</BlockSizeX>
#     <BlockSizeY>512</BlockSizeY>
#     <BandsCount>3</BandsCount>
#     <MaxConnections>10</MaxConnections>
# </GDAL_WMS>